In [ ]:
# Function create an environment with a random mathematical expression
def generateRandom():
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! flexpytester --generate -e symbols -s generated.py -i inputs.csv -o outputs.csv --prefix -t {data_type} --config decayFactor=3 > /dev/null
	! bmhelper apply > /dev/null 
	! make show > /dev/null
	prefix = getPrefix(data_type)
	if unsigned_base:
		csv2sequenceunsigned('inputs.csv','inputs.seq',prefix)
		# csv2sequenceunsigned('outputs.csv','outputs.seq',prefix)
	else:
		csv2sequence('inputs.csv','inputs.seq',prefix)
		# csv2sequence('outputs.csv','outputs.seq',prefix)
# generateRandom()

In [ ]:
# Function to generate the test cases of an existing mathematical expression
def generateExistent(functionFile):
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! flexpytester --compute -e generated.py -i inputs.csv -o outputs.csv --prefix -t {data_type} > /dev/null 2>&1
	! bmhelper apply > /dev/null 2>&1
	! make show > /dev/null 2>&1
	prefix = getPrefix(data_type)
	if unsigned_base:
		csv2sequenceunsigned('inputs.csv','inputs.seq',prefix)
		# csv2sequenceunsigned('outputs.csv','outputs.seq',prefix)
	else:
		csv2sequence('inputs.csv','inputs.seq',prefix)
		# csv2sequence('outputs.csv','outputs.seq',prefix)
# generateExistent()

In [ ]:
# Function to generate the test cases of an existing single neuron within the library
def generateNeuron(library, neuron, seq, benchcore, data_type="float32"):
	! echo "SOURCE_BASM=out.basm" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_BASM=out.basm" > source.mk
	! echo "SIMBATCH_ARGS+= -data-type {data_type}" >> source.mk
	! rm -f notok
	if benchcore:
		! echo "BENCHCOREV2_FILE=sicv2" >> source.mk
		! echo "SIMBATCH_ARGS+=-b" >> source.mk
		appflavor="cpynqapibenchv2"
		if unsigned_base:
			appflavor="cpynqapiunsignedbenchv2"
		! fragtester -neuron-lib-path {library} -fragment-file {neuron}.basm -save-expression generated.py -seq {seq} -create-bmapi bmapi.json -build-app -app-flavor {appflavor} -app-file expression.c -sicv2-endpoints sicv2 -data-type {data_type} || touch notok > /dev/null
	else:
		appflavor="cpynqapi"
		if unsigned_base:
			appflavor="cpynqapiunsigned"
		! fragtester -neuron-lib-path {library} -fragment-file {neuron}.basm -save-expression generated.py -seq {seq} -create-bmapi bmapi.json -build-app -app-flavor {appflavor} -app-file expression.c -data-type {data_type} || touch notok > /dev/null
	if os.path.exists("notok"):
		return False
	! flexpytester --compute -e generated.py -i inputs.csv -o outputs.csv --prefix -t {data_type} > /dev/null 2>&1
	! bmhelper apply > /dev/null 2>&1
	! make hdl > /dev/null 2>&1
	! make show > /dev/null 2>&1
	prefix = getPrefix(data_type)
	if unsigned_base:
		csv2sequenceunsigned('inputs.csv','inputs.seq',prefix)
		# csv2sequenceunsigned('outputs.csv','outputs.seq',prefix)	
	else:
		csv2sequence('inputs.csv','inputs.seq',prefix)
		# csv2sequence('outputs.csv','outputs.seq',prefix)
	return True
# generateNeuron("library", "cosargreal", "0")

In [ ]:
# Function to generate the test cases of an existing single neuron within the library
def getNeuronSeq(library, neuron, data_type):
	! rm -f notok
	! fragtester -neuron-lib-path {library} -fragment-file {neuron}.basm -save-expression generated.py -data-type {data_type} || touch notok > /dev/null
	if os.path.exists("notok"):
		return "0"
	! fragtester -neuron-lib-path {library} -fragment-file {neuron}.basm -save-expression generated.py -describe -data-type {data_type} | grep "Sequences" | cut -d' ' -f2 > seqs.txt
	seq=1
	with open("seqs.txt", "r") as f:
		lines = f.readlines()
		for line in lines:
			seq=line.strip()
			return seq